# Sorting Oddity Files (.ps1)

This is used to sort raw oddity files after testing. It will find the matching txt and edat2 file assumed to be done in the same session and create a folder for it. 


In [ ]:
$files = Get-ChildItem -Path .\* -Include *.edat2, *.txt -File

$groupedFiles = $files | Group-Object { $_.BaseName }

$sortedFileCount = 0
$sortedFolderCount = 0
$missingFiles = @()

foreach ($group in $groupedFiles) {
    $folderName = $group.Name
    $directoryPath = Join-Path -Path (Get-Location) -ChildPath $folderName

    if (-not (Test-Path $directoryPath)) {
        $hasTxt = $group.Group.Extension -contains '.txt'
        $hasEdat2 = $group.Group.Extension -contains '.edat2'

        if ($hasTxt -and $hasEdat2) {
            New-Item -Path $directoryPath -ItemType Directory
            foreach ($file in $group.Group) {
                $newPath = Join-Path -Path $directoryPath -ChildPath $file.Name
                Move-Item -Path $file.FullName -Destination $newPath
                $sortedFileCount++
            }
            $sortedFolderCount++
        } else {

            if (-not $hasTxt) { $missingFiles += "$folderName.txt" }
            if (-not $hasEdat2) { $missingFiles += "$folderName.edat2" }
        }
    }
}

if ($missingFiles.Count -gt 0) {
    Write-Output "Missing Files:"
    $missingFiles | ForEach-Object { Write-Output $_ }
}

Write-Output "Total Files Sorted: $sortedFileCount"
Write-Output "Total Folders Created: $sortedFolderCount"